In [10]:
import os

### Setup if you are running this in Colab

#### Clone repo code

In [ ]:
!git clone https://github.com/OmdenaAI/SaoPauloBrazilChapter_BrazilianSignLanguage

#### Mount your Google Drive folder that has the necessary data

In [ ]:
from google.colab import drive
drive.mount(f'/content/drive/')

In [ ]:
drive_folder = 'Omdena LIBRAS SLP'

#### Check the files

In [ ]:
os.listdir(f'drive/MyDrive/{drive_folder}/Data')

In [ ]:
os.listdir(f'drive/MyDrive/{drive_folder}/Data/preprocessed')

In [ ]:
os.listdir(f'drive/MyDrive/{drive_folder}/Data/preprocessed/landmarks')

#### Copy the data to the Colab environment repo

In [11]:
data_version = 'v4'

required_files = [
]

required_folders = [
    f'/landmarks/{data_version}',
]

print(required_files + required_folders)

['/landmarks/v4']


In [ ]:
import shutil
source_preprocessed_dir = f'drive/MyDrive/{drive_folder}/Data/preprocessed'
target_preprocessed_dir = 'SaoPauloBrazilChapter_BrazilianSignLanguage/data/preprocessed'

In [ ]:
for required_file in required_files:
  source = f'{source_preprocessed_dir}/{required_file}'
  target = f'{target_preprocessed_dir}/{required_file}'
  print(f'Copying file {required_file}')
  print(f'\tFrom: {source}')
  print(f'\tTo: {target}')
  shutil.copy(source, target, dirs_exist_ok=True)

for required_folder in required_folders:
  source = f'{source_preprocessed_dir}/{required_folder}'
  target = f'{target_preprocessed_dir}/{required_folder}'
  print(f'Copying folder {required_folder}')
  print(f'\tFrom: {source}')
  print(f'\tTo: {target}')
  shutil.copytree(source, target, dirs_exist_ok=True)

### Set the current working directory

In [1]:
os.getcwd()

'/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/code'

Current working directory should be the root directory of the project

In [2]:
on_colab = True
if on_colab:
  os.chdir('SaoPauloBrazilChapter_BrazilianSignLanguage')
else:
  os.chdir('..')
  
os.getcwd()

'/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage'

### Set up the python environment

This step is optional. If you have already created the virtual environment, you can skip this step.

In [ ]:
os.system('uv sync')

In [ ]:
os.system('uv venv activate')

### Set the python path to find our code modules

In [3]:
os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + ':' + os.path.join(os.getcwd(), 'code')

In [4]:
print(os.environ['PYTHONPATH'])

:/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/code


### Monitor the training progress live with TensorBoard

In [5]:
%load_ext tensorboard

The magic command below works on Colab or Jupyter Notebook.\
In VSCode, you can use the command palette to open the TensorBoard extension: `>Python: Launch TensorBoard`

In [ ]:
%tensorboard --logdir modelling/runs

### Confirm the experiment settings

In [7]:
import yaml
from pprint import pprint

Below is the main config file for the experiment - `train_config.yaml`\
This is pointing to many other config files, that specify the model, dataset, features, etc.

In [11]:
# load yaml
path = os.path.join(os.getcwd(), 'code','model', 'configs', 'train_config.yaml')
with open(path, 'r') as f:
    config = yaml.safe_load(f)
pprint(config)

{'defaults': ['_self_',
              {'optimizer': 'adam'},
              {'scheduler': 'plateau'},
              {'model': 'lstm'},
              {'dataset': 'dataset'},
              {'training': 'training'},
              {'features': ['positions',
                            'angles',
                            'differences',
                            'distances',
                            'metadata']},
              {'augmentation': 'base_augs'}],
 'general': {'author': None, 'enable_saving': True, 'experiment_name': None},
 'hydra': {'job': {'chdir': False},
           'run': {'dir': 'modelling/outputs/${now:%Y-%m-%d}/${now:%H-%M-%S}'}}}


For some high level changes, just editing the `train_config.yaml` is enough.\
For more detailed changes, you can edit the other config files.\
For example, the `training.yaml` file specifies the training parameters.

In [8]:
# load yaml
path = os.path.join(os.getcwd(), 'code','model', 'configs', 'training', 'training.yaml')
with open(path, 'r') as f:
    config = yaml.safe_load(f)
pprint(config)

{'batch_size': 256,
 'device': 'cuda',
 'k_folds': 5,
 'lr': '1e-4',
 'num_epochs': 300,
 'patience': 50,
 'type': 'cross_validation'}


### Start the training

- The process will run for the `num_epochs` specified in the `training.yaml` file.
- If early stopping is enabled, it will stop when the validation loss stops improving with the `patience` parameter specified.
- The training logs will be saved to the `modelling/runs` directory.
- You can monitor the training progress live with TensorBoard.

In [ ]:
os.system('python code/model/trainer.py training.device=cpu')